In [15]:
import pandas as pd
import math

In [27]:
import requests
from bs4 import BeautifulSoup

#web crawler apartments.com 
headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}
url_base = "https://www.apartments.com/chicago-il/"

for i in range(1, 29):
    url = url_base + str(i)
    response = requests.get(url, headers=headers)
    response_text = response.text
    
    soup = BeautifulSoup(response_text, "html.parser")


utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8
utf-8


In [13]:
#load crime data
columns = ["CASE#", "LATITUDE", "LONGITUDE"]
crime = pd.read_csv("Crimes_Data.csv", usecols=columns)
crime = crime.dropna()
crime

CASE#           0
LATITUDE     2884
LONGITUDE    2884
dtype: int64


,CASE#,LATITUDE,LONGITUDE
2,JE266628,41.748486,-87.602675
3,JE266536,41.880661,-87.731186
5,JE267466,41.871540,-87.705839
6,JE266473,41.780851,-87.649674
7,JE267222,41.859989,-87.735995
...,...,...,...
205762,JE492545,41.902703,-87.629950
205763,JF103181,41.731960,-87.636205
205764,JF102974,41.904582,-87.714655
205765,JF103076,41.987483,-87.712672


In [22]:

#calculate number of crime incidents in the neighborhood\
#from website: https://www.geeksforgeeks.org/program-distance-two-points-earth/#:~:text=For%20this%20divide%20the%20values,is%20the%20radius%20of%20Earth.
def if_crime_in_neighborhood(lo1, la1, lo2, la2, radius):
    lo1 = math.radians(lo1)
    la1 = math.radians(la1)
    lo2 = math.radians(lo2)
    la2 = math.radians(la2)

    lo_dist = lo1 - lo2
    la_dist = la1 - la2

    a = math.sin(la_dist/2)**2 + math.cos(la1) * math.cos(la2) * math.sin(lo_dist/2)**2

    c = 2 * math.asin(math.sqrt(a))

    earth_r_in_mile = 3956

    if c * earth_r_in_mile <= radius:
        return True
    else:
        return False


def find_lo_range(lo1, la1):
    



In [ ]:
#data visualization